In [11]:
import ncd
from ncd import shape

### Foundations of Neural Circuit Diagram Algebraic Operations
*Neural Circuit Diagrams* are constructed from terms called ``Shape``s which represent either data types such as $\mathbb{R}^{a \times c} \times \mathbb{R}^{x}$ or operations such as $f: \mathbb{R}^{a \times b} \rightarrow \mathbb{R}^{c}$. For each data type $a$, there is an identity operation $a: a \rightarrow a$. This means that data types can be uniquely identified by their identity operation.

In [12]:
# The ``shape`` function allows shapes to be built from strings.
#   A, B     -> Tuple of A, B (displayed as (A, B))
#   A B      -> R^(A B)       (displayed as [A B→])
#   A^       -> R^A           (displayed as [A→])
a = shape('a b, c')
f = shape('f:a b -> c^')
print('Type:                ', repr(a))
print('Operation:           ', repr(f))

Type:                 ([a b→], c)
Operation:            [a b→]--f->[c→]


In [13]:
# Shapes can be used to construct new shapes through various operations.
# These operations are often functorial - accepting both data types and
# operations, while preserving composition.

# Cartesian product (+) - this assembles shapes into tuples.
#   As the first expression is a shape, the second is converted from a string to a shape.
print('Cartesian Type:      ', repr(f + 'x'))
#   Combining operations gives an operation which acts on independent segments.
print('Cartesian Operation: ', repr(f + 'g:x -> y'))

Cartesian Type:       ([a b→], x)--(f, x)->([c→], x)
Cartesian Operation:  ([a b→], x)--(f, g)->([c→], y)


In [14]:
# Hom-functor / Lifting (>> / <<) - this lifts a shape, taking (b >> R^a) = R^(b a)
#   We use EXPONENT >> BASE to add to lift to the left,
#       and BASE << EXPONENT to lift to the right.
#       This lets us clearly indicate the axes on which f is applied.
#   Note that R^x (displayed as '[x→]') is different to 'x'. 'x' represents a generic
#   axis, typically of some integer size.
print('Lifting to the right:  ', repr(f << 'x'))
print('Lifting from the left: ', repr(shape('x') >> f))

#   Lifting by an operation yields a natural transformation, an operation on the indexes.
#   The lifting operation's types are reversed. 
#       (g: x -> y) >> (f: [a b→] -> c^) = ([g→f]: y a b -> x c)
print('Lifting by an Operation:', repr(shape('g:x->y') >> f))

Lifting to the right:   [a b x→]--[f←x]->[c x→]
Lifting from the left:  [x a b→]--[x→f]->[x c→]
Lifting by an Operation: [y a b→]--[g→f]->[x c→]


In [15]:
# Algebraic rules are often distributive.
#   Lifting spreads over Cartesian products.
ab, c = shape('a b'), shape('c^')
print(ab + c)
x = shape('x')
print(x >> (ab + c))

([a b→], [c→])
([x a b→], [x c→])


In [16]:
# Configurables are generic shapes. When composed with another shape, a configuration
# is generated which aligns the configurable with that shape. This allows sizes to 
# be contextually derived.
#   When there are multiple configurables with the same name, they are displayed
# with their unique key.
m = shape('*m^')
m2 = shape('*m^')
print(repr(m))
h = ncd.Variant('h', m, m)
print(repr(h))
print(repr(f @ h))

[m=m.AE→]
[m=m.AE→]--h->[m=m.AE→]
[a b→]--f->[c→]--h->[c→]


In [17]:
# In addition to configurables being aligned on composition, various other
#   broadcasting rules exist, modifying shapes to allow them to compose.

# If we compose a non-Cartesian shape with a Cartesian product,
#   then we introduce an intermediate Duplicate operation.
h = shape('h: a -> b')
f = shape('f: b -> c')
g = shape('g: b -> d')
print(repr(h @ (f + g)))

a--h->b--Δ2b->(b, b)--(f, g)->(c, d)


In [18]:
# Similarly, a Cartesian shape followed by a singular shape will
#   have the second shape duplicated into a Cartesian product
#   with itself.
h = shape('h: a -> b')
f = shape('f: c -> b')
g = shape('g: b -> d')
print(repr((h + f) @ g))

(a, c)--(h, f)->(b, b)--(g, g)->(d, d)


In [19]:
# f: p -> [x→b] and g: b -> q do not compose, as [x→b] =/= b.
# However, we can lift g by x and make them composable!
f = shape('f: p -> x^b')
g = shape('g: b -> q')
print(repr(f))
print(repr(g))
print(repr(f @ g))

p--f->[x→b]
b--g->q
p--f->[x→b]--[x→g]->[x→q]
